In [59]:
import pickle

In [60]:
train = pickle.load(open('/tmp/citi/train_sample.pckl/train_sample.pckl', 'rb'))

In [61]:
train['delivery']=train['delivery_available'].map(lambda x: 1 if x else 0)

In [62]:
train['delivery_available'].value_counts()

False    99707
True       293
Name: delivery_available, dtype: int64

In [63]:
train['delivery'].value_counts()

0    99707
1      293
Name: delivery, dtype: int64

In [64]:
train['lat'] = train['location'].map(lambda x: x['latitude'])
train['lon'] = train['location'].map(lambda x: x['longitude'])
train['is_verygood'] = train['description'].map(lambda x: 'отличн'  in x.lower() or \
                                                          'perfect' in  x.lower() or \
                                                          'идеал'   in  x.lower())

train['is_free'] = train['description'].map(lambda x: 'бесплатно' in x.lower() or \
                                                      'даром' in x.lower() or \
                                                      'отдам за киндер' in x.lower())

In [65]:
train['is_used'] = train['description'].map(lambda x: 'б/у' in x.lower() or  ' бу ' in x.lower())

In [66]:
train['is_good']= train['description'].map(lambda x : 'хорош' in x.lower())

In [67]:
train['is_new'] = train['description'].map(lambda x: 'новы' in x.lower() or \
                                                     'нова' in x.lower() or \
                                                     'new'  in x.lower())

In [68]:
train['is_iphone'] = (train['description'].map(lambda x: 'iphone' in x.lower() or 'айфон' in x.lower())) | \
                            (train['name'].map(lambda x: 'iphone' in x.lower() or 'айфон' in x.lower()))

In [69]:
data = train[['category', 'subcategory', 'price', 'lat', 'lon', 'delivery', 'is_iphone', 'is_good', 'is_verygood', 'is_new', 'is_free', 'is_used']]

In [70]:
from sklearn.model_selection import train_test_split

In [71]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('price', axis=1),
    data['price'],
    test_size=0.33, random_state=42)

In [72]:
from sklearn.ensemble import RandomForestRegressor

In [73]:
rf = RandomForestRegressor(n_estimators=1000, n_jobs=-1)

In [74]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=-1, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [75]:
from sklearn.metrics import mean_squared_log_error

In [76]:
mean_squared_log_error(y_test, rf.predict(X_test))**0.5
# 2.461184253128399

2.4417664409798245